# Building A LSTM From Scratch

Import the modules.

In [1]:
import numpy as np
from keras.layers import LSTM
from keras.layers import Dense
from collections import Counter
from keras.layers import Flatten
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing.text import one_hot
from keras.layers.recurrent import SimpleRNN
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


Let's consider an example text: "This is an example."

Define the input and output.

In [2]:
#define documents
docs = ['this is','is an']
# define class labels
labels = ['an','example']

The strategy that we'll adopt to build an LSTM is as follows:

* 1. One-hot encode the words.
* 2. Identify the maximum length of input:Pad the rest of input to the maximum length so that all the inputs are of the same length.
* 3. Convert the words in the input into a one-hot-encoded version.
* 4. Convert the words in the output into a one-hot-encoded version.
* 5. Process the input and the output data, then fit the LSTM model.

Preprocess our dataset so that it can be passed to an LSTM.

In the code 
* Identifying all the unique words and their corresponding frequency (counts) in the dataset. 
* Assigning an ID number to each word.

In [3]:
counts = Counter()
for i,review in enumerate(docs+labels):
    counts.update(review.split())
words = sorted(counts, key=counts.get, reverse=True)
vocab_size=len(words)
word_to_int = {word: i for i, word in enumerate(words, 1)}

print word_to_int.

In [4]:
word_to_int

{'is': 1, 'an': 2, 'this': 3, 'example': 4}

Modify the input and output words with their corresponding IDs.

* Appending the ID of each word of an input sentence into a list, thus making the input (encoded_docs) a list of lists.
* Appending the ID of each word of the output into a list.

In [5]:
encoded_docs = []
for doc in docs:
    encoded_docs.append([word_to_int[word] for word in doc.split()])
encoded_labels = []
for label in labels:
    encoded_labels.append([word_to_int[word] for word in label.split()])

In [6]:
print('encoded_docs: ',encoded_docs)
print('encoded_labels: ',encoded_labels)

encoded_docs:  [[3, 1], [1, 2]]
encoded_labels:  [[2], [4]]


We also have to take care of input length while encoding the input. 
* In cases of sentiment analysis, the input text length can vary from one review to another. 
* The neural network expects the input size to be fixed. 
* To tackle this problem, we perform padding on top of the input. 
* Padding ensures that all inputs are encoded to have a similar length. 

In the code, we are passing encoded_docs to the pad_sequences function, which ensures that all the input data points have the same length—which is equal to the maxlen parameter.

# For those parameters that are shorter than maxlen 
* It pads data points with a value of 0 to achieve a total length of maxlen and the zero padding is done pre so it will add pads value of 0, to the left of the original encoded sentence.

In [7]:
# pad documents to a max length of 2 words
max_length = 2
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='pre')
print(padded_docs)

[[3 1]
 [1 2]]


* Now that the input dataset is created, let's preprocess the output dataset so that it can be passed to the model-training step.
* The typical processing for outputs is to make them into dummy values, that is, make a one-hot-encoded version of the output labels.

In [8]:
# processing the output dataset
one_hot_encoded_labels = to_categorical(encoded_labels, num_classes=5)
print(one_hot_encoded_labels)

[[0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]]


Note: Given the output values (encoded_labels) are {2, 4}, the output vectors have a value of 1 at the second and fourth positions respectively.

# Let's build the model.

An LSTM expects the input to be (batch_size, time_steps, and features_per_timestep) in shape. So first needs to reshape the padded_docs input into the following format.

In [9]:
padded_docs = padded_docs.reshape(2,2,1)

* The 'return_sequences' parameter specifies whether we want to obtain the hidden layer values at each time step. A false value for 'return_sequences' specifies that we want the hidden layer output only at the final timestep.

* In a many-to-one task, where there are many inputs (one input in each time step) and outputs, 'return_sequences' will be false, resulting in the output being obtained only in the final time step. An example of this could be the stock price on the next day, given a sequence of historical five-day stock prices.

* In cases where we try to obtain hidden-layer values in each time step, 'return_sequences' will be set to True. An example of this could be machine translation, where there are many inputs and many outputs.

In [10]:
# define the model
embed_length=1
max_length=2
model = Sequential()
model.add(LSTM(1,activation='tanh', return_sequences=False,recurrent_initializer='Zeros',recurrent_activation='sigmoid',input_shape=(max_length,embed_length),unroll=True))

Instructions for updating:
Colocations handled automatically by placer.


Connect the LSTM output to five nodes of the output layer.

We have performed a Dense(5), as there are five possible classes of output (the output of each example has 5 values, where each value corresponds to the probability of it belonging to word ID 0 to word ID 4).

In [11]:
model.add(Dense(5, activation='softmax'))

Compile and summarize the model.

In [12]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 10        
Total params: 22
Trainable params: 22
Non-trainable params: 0
_________________________________________________________________
None


* The number of parameters is 12 in the LSTM layer as there are four gates (forget, input, cell, and output), which results in four weights and four biases connecting the input to the hidden layer. Additionally, the recurrent layer contains weight values that correspond to the four gates, which gives us a total of 12 parameters.
* The dense layer has a total of 10 parameters as there are five possible classes as output, and thus five weights and five biases that correspond to each connection from the hidden layer to the output layer.

Fit the model to predict the output from the input.

In [13]:
model.fit(padded_docs.reshape(2,2,1),np.array(one_hot_encoded_labels),epochs=500)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
2/2 [==============================] - 1s 563ms/step - loss: 0.4616 - acc: 0.8000
Epoch 2/500
2/2 [==============================] - 0s 998us/step - loss: 0.4611 - acc: 0.8000
Epoch 3/500
2/2 [==============================] - 0s 997us/step - loss: 0.4606 - acc: 0.8000
Epoch 4/500
2/2 [==============================] - 0s 2ms/step - loss: 0.4600 - acc: 0.8000
Epoch 5/500
2/2 [==============================] - 0s 998us/step - loss: 0.4594 - acc: 0.8000
Epoch 6/500
2/2 [==============================] - 0s 997us/step - loss: 0.4589 - acc: 0.8000
Epoch 7/500
2/2 [==============================] - 0s 997us/step - loss: 0.4583 - acc: 0.8000
Epoch 8/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4577 - acc: 0.8000
Epoch 9/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4571 - acc: 0.8000
Epoch 10/500
2/2 [==============================] - 0s 996us/step - loss: 0.4565 - acc: 0.8000
Epoch 11/500
2/2 

Epoch 86/500
2/2 [==============================] - 0s 998us/step - loss: 0.4131 - acc: 0.8000
Epoch 87/500
2/2 [==============================] - 0s 997us/step - loss: 0.4126 - acc: 0.8000
Epoch 88/500
2/2 [==============================] - 0s 998us/step - loss: 0.4121 - acc: 0.8000
Epoch 89/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4116 - acc: 0.8000
Epoch 90/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4111 - acc: 0.8000
Epoch 91/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4106 - acc: 0.8000
Epoch 92/500
2/2 [==============================] - 0s 997us/step - loss: 0.4101 - acc: 0.8000
Epoch 93/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4096 - acc: 0.8000
Epoch 94/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4090 - acc: 0.8000
Epoch 95/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4085 - acc: 0.8000
Epoch 96/500
2/2 [==============================] - 0s 1ms/ste

Epoch 173/500
2/2 [==============================] - 0s 998us/step - loss: 0.3751 - acc: 0.8000
Epoch 174/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3747 - acc: 0.8000
Epoch 175/500
2/2 [==============================] - 0s 998us/step - loss: 0.3743 - acc: 0.8000
Epoch 176/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3740 - acc: 0.8000
Epoch 177/500
2/2 [==============================] - 0s 997us/step - loss: 0.3736 - acc: 0.8000
Epoch 178/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3733 - acc: 0.8000
Epoch 179/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3729 - acc: 0.8000
Epoch 180/500
2/2 [==============================] - 0s 997us/step - loss: 0.3726 - acc: 0.8000
Epoch 181/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3722 - acc: 0.8000
Epoch 182/500
2/2 [==============================] - 0s 998us/step - loss: 0.3719 - acc: 0.8000
Epoch 183/500
2/2 [==============================]

2/2 [==============================] - 0s 1ms/step - loss: 0.3496 - acc: 0.8000
Epoch 260/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3494 - acc: 0.8000
Epoch 261/500
2/2 [==============================] - 0s 997us/step - loss: 0.3492 - acc: 0.8000
Epoch 262/500
2/2 [==============================] - 0s 997us/step - loss: 0.3489 - acc: 0.8000
Epoch 263/500
2/2 [==============================] - 0s 998us/step - loss: 0.3487 - acc: 0.8000
Epoch 264/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3484 - acc: 0.8000
Epoch 265/500
2/2 [==============================] - 0s 998us/step - loss: 0.3482 - acc: 0.8000
Epoch 266/500
2/2 [==============================] - 0s 998us/step - loss: 0.3480 - acc: 0.8000
Epoch 267/500
2/2 [==============================] - 0s 998us/step - loss: 0.3478 - acc: 0.8000
Epoch 268/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3475 - acc: 0.8000
Epoch 269/500
2/2 [==============================] - 0s 1ms/st

Epoch 346/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3323 - acc: 0.8000
Epoch 347/500
2/2 [==============================] - 0s 998us/step - loss: 0.3322 - acc: 0.8000
Epoch 348/500
2/2 [==============================] - 0s 2ms/step - loss: 0.3320 - acc: 0.8000
Epoch 349/500
2/2 [==============================] - 0s 998us/step - loss: 0.3319 - acc: 0.8000
Epoch 350/500
2/2 [==============================] - 0s 998us/step - loss: 0.3317 - acc: 0.8000
Epoch 351/500
2/2 [==============================] - 0s 998us/step - loss: 0.3315 - acc: 0.8000
Epoch 352/500
2/2 [==============================] - 0s 997us/step - loss: 0.3314 - acc: 0.8000
Epoch 353/500
2/2 [==============================] - 0s 997us/step - loss: 0.3312 - acc: 0.8000
Epoch 354/500
2/2 [==============================] - 0s 998us/step - loss: 0.3311 - acc: 0.8000
Epoch 355/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3309 - acc: 0.8000
Epoch 356/500
2/2 [===========================

Epoch 433/500
2/2 [==============================] - 0s 997us/step - loss: 0.3203 - acc: 0.8000
Epoch 434/500
2/2 [==============================] - 0s 998us/step - loss: 0.3202 - acc: 0.8000
Epoch 435/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3201 - acc: 0.8000
Epoch 436/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3200 - acc: 0.8000
Epoch 437/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3199 - acc: 0.8000
Epoch 438/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3198 - acc: 0.8000
Epoch 439/500
2/2 [==============================] - 0s 998us/step - loss: 0.3197 - acc: 0.8000
Epoch 440/500
2/2 [==============================] - 0s 997us/step - loss: 0.3195 - acc: 0.8000
Epoch 441/500
2/2 [==============================] - 0s 998us/step - loss: 0.3194 - acc: 0.8000
Epoch 442/500
2/2 [==============================] - 0s 1ms/step - loss: 0.3193 - acc: 0.8000
Epoch 443/500
2/2 [==============================]

Extract prediction on the first input data point.

In [14]:
model.predict(padded_docs[0].reshape(1,2,1))

array([[0.06413898, 0.04268872, 0.38673618, 0.05115727, 0.4552788 ]],
      dtype=float32)

Inspect the weights.

In [15]:
model.weights

[<tf.Variable 'lstm_1/kernel:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'lstm_1/recurrent_kernel:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'lstm_1/bias:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(1, 5) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(5,) dtype=float32_ref>]

The above weights gives us an intuition of the order in which weights are presented in the output.

* The kernel represents the weights and recurrent represents the connection of the hidden layer from one step to another.

* A LSTM has weights that connect the input to the hidden layer and also weights that connect the previous time step's hidden layer to the current time step's hidden layer.

* The kernel and bias in the dense_1 layer represent the layer that connects the hidden layer value to the final output

The following line of code gives us the computed values of each of the weights.

In [16]:
model.get_weights()

[array([[0.5866583 , 0.46882355, 0.5307267 , 1.1682754 ]], dtype=float32),
 array([[0.44634107, 0.37472215, 0.31417692, 0.41816187]], dtype=float32),
 array([0.3832207 , 1.3373766 , 0.29879212, 0.38187584], dtype=float32),
 array([[-0.55315286, -1.1231616 ,  0.7899263 , -0.8698494 ,  1.1733266 ]],
       dtype=float32),
 array([-0.3702354 , -0.34576902,  0.40953848, -0.35659516,  0.28241307],
       dtype=float32)]

Update the forget gate in time step 1. This step looks at the input and then provides an estimate of how much of the cell state (memory) known so far is to be forgotten (note the usage of the sigmoid function).

In [17]:
input_t0 = 3
cell_state0 = 0
forget0 = input_t0*model.get_weights()[0][0][1] + model.get_weights()[2][1]
forget1 = 1/(1+np.exp(-(forget0)))

Update the cell state based on the updated forget gate. The output of the previous step is being used here to direct the amount of values to be forgotten from the cell state (memory).

In [18]:
cell_state1 = forget1 * cell_state0

Update the input gate value in time step 1. This step gives an estimate of how much new information is to be injected into the cell state based on the current input.

In [19]:
input_t0_1 = input_t0*model.get_weights()[0][0][0] + model.get_weights()[2][0]
input_t0_2 = 1/(1+np.exp(-(input_t0_1)))

Update the cell state based on the updated input value. This is the step where the output from the previous step is being used to dictate the amount of information update that is to happen to cell state (memory).

In [20]:
input_t0_cell1 = input_t0*model.get_weights()[0][0][2] + model.get_weights()[2][2]
input_t0_cell2 = np.tanh(input_t0_cell1)

* The preceding tanh activation helps to determine whether the update from the input will add to or subtract from the cell state (memory). This provides an additional lever, as, if certain information is already conveyed in the current time step and is not useful in future time steps, we are better off wiping it off from the cell state so that this extra information (which might not be helpful in the next step) is wiped from memory

In [21]:
input_t0_cell3 = input_t0_cell2*input_t0_2
input_t0_cell4 = input_t0_cell3 + cell_state1

Update the output gate. This step provides an estimate of how much information will be conveyed in the current time step (note the usage of the sigmoid function in this regard).

In [22]:
output_t0_1 = input_t0*model.get_weights()[0][0][3] + model.get_weights()[2][3]
output_t0_2 = 1/(1+np.exp(-output_t0_1))

Calculate the hidden layer value at time step 1. Note that the final hidden-layer value at a time step is a combination of how much memory and output in the current time step is used to convey for a single time step.

In [23]:
hidden_layer_1 = np.tanh(input_t0_cell4)*output_t0_2

* We are done calculating the hidden layer value output from the first time step. In the next steps, we will pass the updated cell state value from time step 1 and the output of the hidden layer value from time step 1 as inputs to time step 2

Pass the input value at time step 2 and the cell state value going into time step 2

In [24]:
input_t1 = 1
cell_state1 = input_t0_cell4

Update the forget gate value

In [25]:
forget21 = hidden_layer_1*model.get_weights()[1][0][1] + model.get_weights()[2][1] + input_t1*model.get_weights()[0][0][1]
forget_22 = 1/(1+np.exp(-(forget21)))

Update the cell state value in time step 2

In [26]:
cell_state2 = cell_state1 * forget_22
input_t1_1 = input_t1*model.get_weights()[0][0][0] + model.get_weights()[2][0] + hidden_layer_1*model.get_weights()[1][0][0]
input_t1_2 = 1/(1+np.exp(-(input_t1_1)))
input_t1_cell1 = input_t1*model.get_weights()[0][0][2] + model.get_weights()[2][2]+ hidden_layer_1*model.get_weights()[1][0][2]
input_t1_cell2 = np.tanh(input_t1_cell1)
input_t1_cell3 = input_t1_cell2*input_t1_2
input_t1_cell4 = input_t1_cell3 + cell_state2

Update the hidden layer output value based on the combination of updated cell state and the magnitude of output that is to be made.

In [27]:
output_t1_1 = input_t1*model.get_weights()[0][0][3] + model.get_weights()[2][3]+ hidden_layer_1*model.get_weights()[1][0][3]
output_t1_2 = 1/(1+np.exp(-output_t1_1))
hidden_layer_2 = np.tanh(input_t1_cell4)*output_t1_2

Pass the hidden layer output through the dense layer.

In [28]:
final_output = hidden_layer_2 * model.get_weights()[3][0] + model.get_weights()[4]

Run softmax on top of the output we just obtained.

In [29]:
np.exp(final_output)/np.sum(np.exp(final_output))

array([0.06413898, 0.04268872, 0.38673618, 0.05115727, 0.45527884],
      dtype=float32)

We can notice that the output obtained here is exactly the same as what we obtained from the model.predict method.